# SpotlightAI Batch Demo
Process a folder of OTT assets and produce promos using the agentic workflow.

In [ ]:

import os, asyncio, glob
from pathlib import Path
from dotenv import load_dotenv
load_dotenv()

from src.graph.builder import build_graph
from src.graph.state import PromoState

INPUT_DIR = os.getenv("BATCH_INPUT_DIR", "/path/to/ott/assets")  # set in .env for convenience
AUDIENCES = (os.getenv("BATCH_AUDIENCES","genpop")).split(",")
TARGET_FORMATS = (os.getenv("BATCH_TARGET_FORMATS","9:16,16:9")).split(",")

vids = []
for ext in ("*.mp4","*.mkv","*.mov"):
    vids.extend(glob.glob(str(Path(INPUT_DIR)/ext)))
print(f"Found {len(vids)} videos")

graph = build_graph()

async def process_one(path):
    # Using local path URI style for demo; storage adapter can fetch remote URIs too.
    state = PromoState(
        asset_uri=f"local://{path}",
        campaign_id="cmp_batch",
        audiences=AUDIENCES,
        target_formats=TARGET_FORMATS,
        show_title=Path(path).stem,
        logline="Batch processed asset."
    )
    # Storage.get() maps any URI to local cache; for local://, just copy/alias if you extend it.
    out = await graph.ainvoke(state)
    print(Path(path).name, "->", out.published_uri, out.metadata.get("ar_variants"))

async def main():
    for v in vids:
        await process_one(v)

await main()
